In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import json
import tensorflow_text as tf_text

In [28]:
with open("E:/MLCourse/foxBot/foxPython/ChatBot/intentsVN.json",encoding="utf-8") as file:
    data=json.load(file)
patternlist=[]
labels=[]
for dt in data["intents"]:
    for pattern in dt["patterns"]:
        patternlist.append(pattern)
        labels.append(dt["tag"])
labels
labels=np.array(labels)
labelsOH=pd.get_dummies(labels)

# OLD VER
# with open("E:/MLCourse/TF2/Notebook/SmartFennec/bi_grams.txt",encoding="utf-8") as f:
#     bi_gramtxt=f.read()
# bi_gram_list=[]
# for each in bi_gramtxt.split(","):
#     each=each[:-1]
#     each=each[2:]
#     bi_gram_list.append(each)

# with open("E:/MLCourse/TF2/Notebook/SmartFennec/vietnamese-stopwords.txt",encoding="utf-8") as f:
#     stopword_txt=f.readlines()
    
# stopword_list=[]
# for each in stopword_txt:
#     stopword_list.append(each[:-1])


In [29]:
VNTokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=20000,oov_token="<UNK>")
VNTokenizer.fit_on_texts(patternlist)
pattern_tokenized=VNTokenizer.texts_to_sequences(patternlist)
pattern_preprocessed=tf.keras.preprocessing.sequence.pad_sequences(pattern_tokenized)
mlen=pattern_preprocessed.shape[1]
tokens_size=len(VNTokenizer.word_index)

In [30]:
with open("E:/MLCourse/foxBot/foxPython/word_2_vec/data/new_vocabs_weights.json","r",encoding="utf-8") as f:
    vocab_weights=json.load(f)
len(vocab_weights)

5227

In [31]:
num_tokens = tokens_size+1
embedding_dim = 128
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in VNTokenizer.word_index.items():
    embedding_vector = vocab_weights.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

    

Converted 79 words (8 misses)


In [32]:
embedding_layer = tf.keras.layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [33]:
def LSTMModel():
    int_sequences_input = tf.keras.Input(shape=(None,), dtype="int64")
    x=embedding_layer(int_sequences_input)
    x=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True,))(x)
    x=tf.keras.layers.GlobalMaxPool1D()(x)
    x=tf.keras.layers.Dense(16,activation="relu")(x)
    x=tf.keras.layers.Dense(10,activation="softmax")(x)
    return tf.keras.Model(int_sequences_input,x)
    

In [34]:
model=LSTMModel()
model.compile(tf.keras.optimizers.Adam(learning_rate=0.02),loss="categorical_crossentropy",metrics="categorical_accuracy")

In [35]:
model.fit(pattern_preprocessed,labelsOH,batch_size=16,epochs=500)
tagModel.summary()

Epoch 1/500
4/4 [==============================] - 3s 8ms/step - loss: 2.2366 - categorical_accuracy: 0.2031
Epoch 2/500
4/4 [==============================] - 0s 10ms/step - loss: 1.3848 - categorical_accuracy: 0.5938
Epoch 3/500
4/4 [==============================] - 0s 9ms/step - loss: 0.6999 - categorical_accuracy: 0.8281
Epoch 4/500
4/4 [==============================] - 0s 8ms/step - loss: 0.2756 - categorical_accuracy: 0.9531
Epoch 5/500
4/4 [==============================] - 0s 8ms/step - loss: 0.1328 - categorical_accuracy: 0.9688
Epoch 6/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0882 - categorical_accuracy: 0.9688
Epoch 7/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0365 - categorical_accuracy: 0.9844
Epoch 8/500
4/4 [==============================] - 0s 8ms/step - loss: 0.1137 - categorical_accuracy: 0.9688
Epoch 9/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0306 - categorical_accuracy: 0.9844
Epoch 10/500
4/4 [

In [37]:
tagModel=tf.keras.models.load_model("E:/MLCourse/foxBot/foxPython/EntitiesTagger/TagModel")

In [38]:
tagdict={1: '<UNK>',
 2: 'o',
 3: 'B-mh',
 4: 'B-name',
 5: 'B-obj',
 6: 'I-mh',
 7: 'I-obj',
 8: 'B-adj',
 9: 'B-v',
 10: 'I-name',
 11: 'B-num',
 12: 'I-location',
 13: 'I-adj',
 14: 'B-location',
 15: 'B-time',
 16: 'I-v',
 0: '<PAD>'}

In [39]:
with open("E:/MLCourse/foxBot/foxPython/EntitiesTagger/tagFolder/tagtrain.json","r",encoding="utf-8") as f:
    data=json.load(f)
textlist=[]

for each in data:
    textlist.append(each["sentence"])

TextTokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=20000,lower=True,oov_token="<UNK>")
TextTokenizer.fit_on_texts(textlist)

tokenizedText=TextTokenizer.texts_to_sequences(textlist)
textFinal=tf.keras.preprocessing.sequence.pad_sequences(tokenizedText)


def insert_text(text):
    text_tokenized=TextTokenizer.texts_to_sequences([text])
    finaltext=tf.keras.preprocessing.sequence.pad_sequences(text_tokenized,maxlen=textFinal.shape[1])
    return finaltext


In [40]:
def entitiesOutput(rs):
    txt=[]
    for i in rs[0]:
        txt.append(tagdict[np.argmax(i)])
        # use argmax 
    return txt

In [41]:
labelsre=np.array(list(labelsOH.columns))
def chat():
    print("start")
    while True:
        inp=input("You: ")
        if inp=="quit":
            break
        entitiesPred=np.round(tagModel.predict(insert_text(inp)),decimals=2)
        entities_output=entitiesOutput(entitiesPred)
        inp_tokenized=VNTokenizer.texts_to_sequences([inp])
        inp_preprocessed=tf.keras.preprocessing.sequence.pad_sequences(inp_tokenized,maxlen=mlen)
        rs=np.round(model.predict(inp_preprocessed),decimals=2)
        print("You ask: ",inp)
        print("Sequence: ",inp_preprocessed)
        print("entities: ",entities_output)
        print("raw output: ",rs)
        print("output: ",labelsre[np.argmax(rs[0])])
        print("")
        
chat()

start
You ask:  hello cậu
Sequence:  [[ 0  0  0  0  0  0  0 53  1]]
entities:  ['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 'o']
raw output:  [[0.   0.   0.   0.   0.47 0.53 0.   0.   0.   0.  ]]
output:  greeting

You ask:  Giúp mình bài này nha
Sequence:  [[ 0  0  0  0 20  7  1 21 41]]
entities:  ['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 'o', 'o', 'o', 'o', 'B-mh']
raw output:  [[0.   0.01 0.   0.   0.99 0.   0.   0.   0.   0.  ]]
output:  goodbye

You ask:  Giúp mình bài này được không
Sequence:  [[ 0  0  0 20  7  1 21 11  6]]
entities:  ['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 'o', 'o', 'o', 'o', 'o', 'o']
raw output:  [[0. 1. 

issue: (1)khi được hỏi, bot sẽ cư xử sao ?
        (2) Khi nào mình sẽ đưa data vào brain để sử dụng sau
        (3) Structure trả lời sẽ như thế nào để phù hợp với (1)
        (4) Limit của cấu trúc này là bot sẽ rất bị động
        (5) intents sao mà Bot đạt tối ưu nhất có thể
        (6) Nếu entities liên kết với nhau thì mình sẽ có gì
        (7) Bot decide được không

(1)Solution:
Mỗi situation sẽ có một dictionary để fully kích hoạt
{asking -> {@chủ_đề->[@main, @vấn_đề}]}
Ví dụ: {askingme -> {studying->[môn học, vấn đề được hỏi}]}

(2)Tạm thời là mọi thông tin (dưới dạng dictionary), để search thông tin thì mình sẽ đo sự tương quan giữa hai câu (có thể từ entities)

(3) có thể sẽ liên quan đến data của (5)

(4) có thể random hỏi user nếu user có đặc điểm nào đó đáng lưu ý

(7) Environment: Chat dialogue ,Action: Chat với người dùng, Reward: user's sentiment